In [ ]:
import numpy as np 
import tensorflow as tf 
from tensorflow.keras.models import Model ,Sequential
from tensorflow.keras.layers import Dense,Conv2D,Dropout,MaxPool2D,BatchNormalization,Input,\
Flatten,GlobalAveragePooling2D
import math 
import warnings 
from tensorflow.keras import regularizers
warnings.filterwarnings("ignore")
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import Sequence
from tensorflow.keras.applications import VGG16 , EfficientNetB7
import cv2 as cv
from collections import defaultdict
import os 
import shutil
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix , classification_report
import pandas as pd
import seaborn as sns 

## Data preparation :

## VGG16 :

In [ ]:
vgg = EfficientNetB7(weights="imagenet",include_top=False ,input_shape=(72,72,3),pooling="avg")

In [ ]:
def build_vgg() : 
 
 for layer in vgg.layers[-6:] :
    layer.trainable = True
 x = vgg.output
 #x = Flatten()(x)
 x = BatchNormalization()(x)
 x = Dropout(0.25)(x)
 predictions = Dense(512,activation="relu",kernel_regularizer=regularizers.l2(0.0001))(x)
 predictions= BatchNormalization()(predictions)
 #predictions = Dropout(0.25)(predictions)
 #predictions = Dense(256,activation="relu")(predictions)
 #predictions = BatchNormalization()(predictions)
 predictions = Dropout(0.25)(predictions)
 predictions = Dense(128,activation="relu",kernel_regularizer=regularizers.l2(0.0001))(predictions)
 #predictions = BatchNormalization()(predictions)
 #predictions = Dropout(0.25)(predictions)
 #predictions = Dense(64,activation="relu")(predictions)
 #predictions = BatchNormalization()(predictions)
 #predictions = Dropout(0.25)(predictions)
 #predictions = Dense(32,activation="relu")(predictions)
 #predictions = BatchNormalization()(predictions)
 predictions = Dropout(0.25)(predictions)
 predictions = Dense(7,activation="softmax")(predictions)
 model_VGG = Model(vgg.input,predictions)
 
 model_VGG.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss=[tf.keras.losses.CategoricalCrossentropy()],metrics=["accuracy"])
 return model_VGG

In [ ]:
from keras.utils.vis_utils import plot_model
model = build_vgg()
plot_model(model, show_shapes=True, show_layer_names=True, expand_nested=True, dpi=50, to_file='model.png')

In [ ]:
print(model.summary())

In [ ]:
val_generator = ImageDataGenerator(rescale=1/255)

tr_generator = ImageDataGenerator(rescale = 1/255,
                                  zoom_range=0.3,
                                  shear_range=0.3,
                                  horizontal_flip=True,
                                  rotation_range=30,
                                  fill_mode="nearest")

In [ ]:
val_data = val_generator.flow_from_directory("../input/fer2013/test",
                                             batch_size= 64,
                                             target_size=(72,72),
                                             color_mode="rgb",
                                             interpolation="nearest")
tr_data = tr_generator.flow_from_directory("../input/fer2013/train",
                                          batch_size= 64,
                                          target_size=(72,72),
                                          color_mode="rgb",
                                          interpolation="nearest")

In [ ]:
model_VGG = build_vgg()
vgg_step_1 = ModelCheckpoint("vgg_step1.h5",monitor="val_accuracy",mode="max",save_best_only=True,
                            save_weights_only=True,verbose=1,period=1)
#earlyStoppingCallback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=EARLY_STOPPING_CRITERIA,verbose= 1 ,restore_best_weights=True)

In [ ]:
history=model_VGG.fit(tr_data,validation_data=val_data,epochs=100,callbacks=[vgg_step_1])

In [ ]:
plot_function(history)

In [ ]:
val_data = val_generator.flow_from_directory("../input/fer2013/test",
                                             shuffle = False,
                                             batch_size= 64,
                                             target_size=(72,72),
                                             color_mode="rgb",
                                             interpolation="nearest")

In [ ]:
y_true = val_data.classes

In [ ]:
fernet_model = build_vgg()
fernet_model.load_weights("vgg_step1.h5")

In [ ]:
y_pred = fernet_model.predict(val_data)

In [ ]:
y_pred = np.argmax(y_pred,axis=1)

In [ ]:
classes = list(val_data.class_indices.keys())

In [ ]:
fig = plt.figure(figsize = (15,15))
cf = pd.DataFrame(confusion_matrix(y_true,y_pred),columns=classes,index=classes)
sns.heatmap(cf,annot=True,square=True)

In [ ]:
print(classification_report(y_true,y_pred,target_names=classes))

In [ ]:
import plotly.express as px
x = px.line(data_frame= history , y= ["accuracy" , "val_accuracy"] ,markers = True )
x.update_xaxes(title="Number of Epochs")
x.update_yaxes(title = "Accuracy")
x.update_layout(showlegend = True,
    title = {
        'text': 'Accuracy vs Number of Epochs',
        'y':0.94,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
x.show()